<a href="https://colab.research.google.com/github/yashveersinghsohi/Hospital_Mortality_Prediction/blob/master/Hospital_00_Preliminary_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns